In [1]:
!pip install tensorflow-determinism

In [2]:
!rm -rf weight.hdf55

In [3]:
!rm -rf videos

In [4]:
!rm -rf validation

In [5]:
!rm train_new.csv

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
import cv2
import math
from glob import glob
import numpy as np
import tensorflow as tf
import random as python_random
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, LSTM, TimeDistributed
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Nadam  
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import accuracy_score

In [8]:
os.environ['PYTHONHASHSEED']= '123'
os.environ['TF_CUDNN_DETERMINISTIC']= '1'

In [9]:
RandomState(MT19937(SeedSequence(28)))

RandomState(MT19937) at 0x7F435A583380

In [10]:
# np.random.seed(123)
python_random.seed(28)
tf.random.set_seed(28)

In [11]:
NAME = 0
TAG = 1
ONLY = 0

In [12]:
!pwd

/content


In [13]:
!cp /content/drive/MyDrive/videos.zip /content/

In [14]:
!cp /content/drive/MyDrive/validation_set.zip /content/

In [15]:
# !cp /content/weight.hdf55_5609 /content/drive/MyDrive/ -rf

In [16]:
!unzip videos.zip -d videos -x "*.txt"

Archive:  videos.zip
  inflating: videos/criss_cross/0a2L7tfx8XbU.mkv  
  inflating: videos/criss_cross/13a2L7tfx8XbU.mkv  
  inflating: videos/criss_cross/14a2L7tfx8XbU.mkv  
  inflating: videos/criss_cross/15a2L7tfx8XbU.mkv  
  inflating: videos/criss_cross/16BSMS3ER8KqU.mkv  
  inflating: videos/criss_cross/17uXqTDoitR8k.mkv  
  inflating: videos/criss_cross/187trwOZKNFCw.mkv  
  inflating: videos/criss_cross/1gzaCxDVQL90.mkv  
  inflating: videos/criss_cross/207trwOZKNFCw.mkv  
  inflating: videos/criss_cross/217trwOZKNFCw.mkv  
  inflating: videos/criss_cross/22o2aamD_J8AM.mkv  
  inflating: videos/criss_cross/23CvBktaDUJhI.mkv  
  inflating: videos/criss_cross/24CvBktaDUJhI.mkv  
  inflating: videos/criss_cross/25CvBktaDUJhI.mkv  
  inflating: videos/criss_cross/26A7GFt-E5gAU.mkv  
  inflating: videos/criss_cross/27A7GFt-E5gAU.mkv  
  inflating: videos/criss_cross/285JNwhyhpoow.mkv  
  inflating: videos/criss_cross/30UDXfEmHyxJw.mkv  
  inflating: videos/criss_cross/31Nf4Fg0ZjfnM

In [17]:
videos_path = './videos'

videos_folder = os.listdir(videos_path)

videos_tags_list = []
for exercise_folder in os.listdir(path=videos_path):
    if not exercise_folder.startswith('.'):  # skip hidden files
        for video in os.listdir(path=f'{videos_path}/{exercise_folder}'):
            videos_tags_list.append({'Name': video, 'Tag': exercise_folder})           
df = pd.DataFrame(data=videos_tags_list)


In [18]:
pd.options.display.max_rows = None

In [19]:
df

,Name,Tag
0,26A7GFt-E5gAU.mkv,criss_cross
1,47SvRSTTl5jII.mkv,criss_cross
2,37QL7Te83ONis.mkv,criss_cross
3,23CvBktaDUJhI.mkv,criss_cross
4,187trwOZKNFCw.mkv,criss_cross
5,395vmNFEOXfBU.mkv,criss_cross
6,31Nf4Fg0ZjfnM.mkv,criss_cross
7,24CvBktaDUJhI.mkv,criss_cross
8,46SvRSTTl5jII.mkv,criss_cross
9,217trwOZKNFCw.mkv,criss_cross


In [20]:
#df = df.drop(index=[*range(43,49)])

In [21]:
df.shape # (117, 2)

(117, 2)

In [22]:
tags = list(df['Tag'].unique())
tags = tags[0:3]

In [23]:
tags

['criss_cross', 'double_leg', 'rollup']

In [24]:
train, test = train_test_split(df, test_size=0.1, stratify=df['Tag'], random_state=28)

In [25]:
train.shape, test.shape # ((105, 2), (12, 2))

((105, 2), (12, 2))

In [26]:
%rm -rf train


In [27]:
os.mkdir(f'{videos_path}'+'/'+"train/")
os.mkdir(f'{videos_path}'+'/'+"test/")

for tag in tags:
  os.mkdir(f'{videos_path}'+'/'+"train/"+ tag)
  os.mkdir(f'{videos_path}'+'/'+"test/"+ tag)


In [28]:
# storing the frames from training videos
count2 = 0
for video in tqdm(train.itertuples(index=False)):
 
  count = 0
  videoFile = video[NAME]
  tag = video[TAG]
  cap = cv2.VideoCapture(f'{videos_path}/'+f'{tag}'+'/'+videoFile)   # capturing the video from the given path
  frameRate = cap.get(5) #frame rate
  x=1
  while cap.isOpened():
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
      break
    if (frameId % math.floor(frameRate) == 0):
      # storing the frames in a new folder named train_1
      
      filename =f'{videos_path}'+'/train/' +f'{tag}/'+f"video{count2}_frame%d.jpg" % count; count+=1
      print(filename)
      cv2.imwrite(filename, frame)
  count2 += 1
cap.release()

0it [00:00, ?it/s]

./videos/train/rollup/video0_frame0.jpg
./videos/train/rollup/video0_frame1.jpg
./videos/train/rollup/video0_frame2.jpg
./videos/train/rollup/video0_frame3.jpg
./videos/train/rollup/video0_frame4.jpg
./videos/train/rollup/video0_frame5.jpg
./videos/train/rollup/video0_frame6.jpg
./videos/train/rollup/video0_frame7.jpg
./videos/train/rollup/video0_frame8.jpg
./videos/train/rollup/video0_frame9.jpg
./videos/train/rollup/video0_frame10.jpg
./videos/train/rollup/video0_frame11.jpg
./videos/train/rollup/video0_frame12.jpg
./videos/train/rollup/video0_frame13.jpg
./videos/train/rollup/video0_frame14.jpg
./videos/train/rollup/video0_frame15.jpg
./videos/train/rollup/video0_frame16.jpg
./videos/train/rollup/video0_frame17.jpg
./videos/train/rollup/video0_frame18.jpg
./videos/train/rollup/video0_frame19.jpg
./videos/train/rollup/video0_frame20.jpg


1it [00:01,  1.96s/it]

./videos/train/rollup/video0_frame21.jpg
./videos/train/criss_cross/video1_frame0.jpg


3it [00:02,  1.82it/s]

...

./videos/train/criss_cross/video102_frame2.jpg
./videos/train/double_leg/video103_frame0.jpg


104it [01:07,  3.00it/s]

./videos/train/double_leg/video103_frame1.jpg
./videos/train/rollup/video104_frame0.jpg
./videos/train/rollup/video104_frame1.jpg
./videos/train/rollup/video104_frame2.jpg
./videos/train/rollup/video104_frame3.jpg
./videos/train/rollup/video104_frame4.jpg
./videos/train/rollup/video104_frame5.jpg


105it [01:07,  1.55it/s]

./videos/train/rollup/video104_frame6.jpg
./videos/train/rollup/video104_frame7.jpg
./videos/train/rollup/video104_frame8.jpg
./videos/train/rollup/video104_frame9.jpg
./videos/train/rollup/video104_frame10.jpg
./videos/train/rollup/video104_frame11.jpg
./videos/train/rollup/video104_frame12.jpg
./videos/train/rollup/video104_frame13.jpg


In [29]:
 
train_image = []
train_class = []

for tag in tags:
    images = glob(f"{videos_path}/train/{tag}/*.jpg")
    print(f"{videos_path}/train/{tag}/*.jpg")
    for i in tqdm(range(len(images))):
        # creating the image name
        train_image.append(images[i])
        train_class.append(tag)


./videos/train/criss_cross/*.jpg


100%|██████████| 84/84 [00:00<00:00, 407779.56it/s]


./videos/train/double_leg/*.jpg


100%|██████████| 112/112 [00:00<00:00, 309257.44it/s]


./videos/train/rollup/*.jpg


100%|██████████| 627/627 [00:00<00:00, 995016.50it/s]


In [30]:
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('train_new.csv',header=True, index=False)

In [31]:
train_data.head(5)

,image,class
0,./videos/train/criss_cross/video99_frame1.jpg,criss_cross
1,./videos/train/criss_cross/video100_frame2.jpg,criss_cross
2,./videos/train/criss_cross/video82_frame1.jpg,criss_cross
3,./videos/train/criss_cross/video50_frame2.jpg,criss_cross
4,./videos/train/criss_cross/video25_frame2.jpg,criss_cross


In [32]:
train_data.shape # (776, 2) / (823, 2)

(823, 2)

In [33]:
train = pd.read_csv('train_new.csv')
train.head()
train

,image,class
0,./videos/train/criss_cross/video99_frame1.jpg,criss_cross
1,./videos/train/criss_cross/video100_frame2.jpg,criss_cross
2,./videos/train/criss_cross/video82_frame1.jpg,criss_cross
3,./videos/train/criss_cross/video50_frame2.jpg,criss_cross
4,./videos/train/criss_cross/video25_frame2.jpg,criss_cross
5,./videos/train/criss_cross/video101_frame2.jpg,criss_cross
6,./videos/train/criss_cross/video102_frame0.jpg,criss_cross
7,./videos/train/criss_cross/video7_frame1.jpg,criss_cross
8,./videos/train/criss_cross/video81_frame1.jpg,criss_cross
9,./videos/train/criss_cross/video67_frame0.jpg,criss_cross


In [34]:
SAMPLES = 0

In [35]:
X = np.empty((train.shape[SAMPLES], 224, 224, 3))
# for loop to read and store frames
for i in tqdm(range(train.shape[SAMPLES])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    # img = img/255
    # appending the image to the train_image list
    X[i] = img

100%|██████████| 823/823 [00:08<00:00, 100.38it/s]


In [36]:

# ## Loading VGG16 model
# base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
# base_model.trainable = False ## Not trainable weights

## Preprocessing input

X = preprocess_input(X) 

In [37]:
    # preprocess_input = keras.applications.inception_v3.preprocess_input
# X = inception_v3.preprocess_input(X)

In [38]:
X.shape # (776, 224, 224, 3)

(823, 224, 224, 3)

In [39]:
y = train['class']

In [40]:
y

0      criss_cross
1      criss_cross
2      criss_cross
3      criss_cross
4      criss_cross
5      criss_cross
6      criss_cross
7      criss_cross
8      criss_cross
9      criss_cross
10     criss_cross
11     criss_cross
12     criss_cross
13     criss_cross
14     criss_cross
15     criss_cross
16     criss_cross
17     criss_cross
18     criss_cross
19     criss_cross
20     criss_cross
21     criss_cross
22     criss_cross
23     criss_cross
24     criss_cross
25     criss_cross
26     criss_cross
27     criss_cross
28     criss_cross
29     criss_cross
30     criss_cross
31     criss_cross
32     criss_cross
33     criss_cross
34     criss_cross
35     criss_cross
36     criss_cross
37     criss_cross
38     criss_cross
39     criss_cross
40     criss_cross
41     criss_cross
42     criss_cross
43     criss_cross
44     criss_cross
45     criss_cross
46     criss_cross
47     criss_cross
48     criss_cross
49     criss_cross
50     criss_cross
51     criss_cross
52     criss

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)


In [42]:
y

0      criss_cross
1      criss_cross
2      criss_cross
3      criss_cross
4      criss_cross
5      criss_cross
6      criss_cross
7      criss_cross
8      criss_cross
9      criss_cross
10     criss_cross
11     criss_cross
12     criss_cross
13     criss_cross
14     criss_cross
15     criss_cross
16     criss_cross
17     criss_cross
18     criss_cross
19     criss_cross
20     criss_cross
21     criss_cross
22     criss_cross
23     criss_cross
24     criss_cross
25     criss_cross
26     criss_cross
27     criss_cross
28     criss_cross
29     criss_cross
30     criss_cross
31     criss_cross
32     criss_cross
33     criss_cross
34     criss_cross
35     criss_cross
36     criss_cross
37     criss_cross
38     criss_cross
39     criss_cross
40     criss_cross
41     criss_cross
42     criss_cross
43     criss_cross
44     criss_cross
45     criss_cross
46     criss_cross
47     criss_cross
48     criss_cross
49     criss_cross
50     criss_cross
51     criss_cross
52     criss

In [43]:
y.shape

(823,)

In [44]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [45]:
y_train.sort_index()

,criss_cross,double_leg,rollup
0,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0
10,1,0,0
11,1,0,0


In [46]:
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model.trainable = False ## Not trainable weights

In [47]:
base_model = InceptionV3(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))
base_model.trainable = False ## Not trainable weights

In [48]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [49]:
# model = Sequential([base_model])
# model.add(Flatten())
# model.add(Dense(1024, activation='relu', input_shape=(25088,)))
# model.add(Dropout(0.5))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(3, activation='softmax'))

In [50]:
# model = Sequential([base_model,
#                     Flatten(),
#                     Dense(50, activation='relu'),
#                     Dense(20, activation='relu'),
#                     Dense(3, activation='softmax')
#                     ])

In [51]:
# model = Sequential([base_model,
#                     Flatten(),
#                     Dense(512, activation="relu"),
#                     Dropout(0.5),
#                     Dense(3, activation="softmax")])

In [52]:
model = Sequential([base_model,
                    Flatten(),
                    Dense(3, activation="softmax")])

In [53]:
X_train.shape

(658, 224, 224, 3)

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 6147      
Total params: 21,808,931
Trainable params: 6,147
Non-trainable params: 21,802,784
_________________________________________________________________


In [55]:
# defining a function to save the weights of best model
mcp_save = ModelCheckpoint('weight.hdf55', save_best_only=True, monitor='val_loss', mode='min')

In [56]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [57]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=5,  restore_best_weights=True)

In [58]:
# training the model
tf.random.set_seed(28)
model.fit( X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[mcp_save, es], batch_size=16,
          shuffle=False)

Epoch 1/20
42/42 [==============================] - 11s 114ms/step - loss: 2.7058 - accuracy: 0.6930 - val_loss: 2.5677 - val_accuracy: 0.4545
INFO:tensorflow:Assets written to: weight.hdf55/assets
Epoch 2/20
42/42 [==============================] - 2s 58ms/step - loss: 2.1720 - accuracy: 0.7553 - val_loss: 2.7428 - val_accuracy: 0.5333
Epoch 3/20
42/42 [==============================] - 2s 50ms/step - loss: 1.2731 - accuracy: 0.8343 - val_loss: 1.2860 - val_accuracy: 0.7515
INFO:tensorflow:Assets written to: weight.hdf55/assets
Epoch 4/20
42/42 [==============================] - 2s 58ms/step - loss: 0.6427 - accuracy: 0.8921 - val_loss: 0.6563 - val_accuracy: 0.8182
INFO:tensorflow:Assets written to: weight.hdf55/assets
Epoch 5/20
42/42 [==============================] - 2s 58ms/step - loss: 0.2978 - accuracy: 0.9286 - val_loss: 0.4687 - val_accuracy: 0.8848
INFO:tensorflow:Assets written to: weight.hdf55/assets
Epoch 6/20
42/42 [==============================] - 2s 52ms/step - loss: 

In [59]:
model.load_weights("weight.hdf55")

In [60]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [61]:
!unzip validation_set.zip

Archive:  validation_set.zip
   creating: validation/criss_cross/
  inflating: validation/criss_cross/0BSMS3ER8KqU.mkv  
  inflating: validation/criss_cross/10fQiCxMwfjX0.mkv  
  inflating: validation/criss_cross/11fQiCxMwfjX0.mkv  
  inflating: validation/criss_cross/13VYDOWnxqqyk.mkv  
  inflating: validation/criss_cross/14VYDOWnxqqyk.mkv  
  inflating: validation/criss_cross/15VYDOWnxqqyk.mkv  
  inflating: validation/criss_cross/16scvPiteYVWw.mkv  
  inflating: validation/criss_cross/17scvPiteYVWw.mkv  
  inflating: validation/criss_cross/18scvPiteYVWw.mkv  
  inflating: validation/criss_cross/1BSMS3ER8KqU.mkv  
  inflating: validation/criss_cross/2BSMS3ER8KqU.mkv  
  inflating: validation/criss_cross/4D2Xp6LmwYPY.mkv  
  inflating: validation/criss_cross/5o2aamD_J8AM.mkv  
  inflating: validation/criss_cross/6CzZnlEf-Y8A.mkv  
  inflating: validation/criss_cross/7fQiCxMwfjX0.mkv  
  inflating: validation/criss_cross/9fQiCxMwfjX0.mkv  
   creating: validation/double_leg/
  inflatin

In [62]:
videos_path = './validation'

videos_folder = os.listdir(videos_path)
videos_tags_list = []
for exercise_folder in os.listdir(path=videos_path):
    if not exercise_folder.startswith('.'):  # skip hidden files
        for video in os.listdir(path=f'{videos_path}/{exercise_folder}'):
            videos_tags_list.append({'Name': video, 'Tag': exercise_folder})           
df_validation = pd.DataFrame(data=videos_tags_list)
df_validation

,Name,Tag
0,4D2Xp6LmwYPY.mkv,criss_cross
1,0BSMS3ER8KqU.mkv,criss_cross
2,11fQiCxMwfjX0.mkv,criss_cross
3,2BSMS3ER8KqU.mkv,criss_cross
4,6CzZnlEf-Y8A.mkv,criss_cross
5,9fQiCxMwfjX0.mkv,criss_cross
6,16scvPiteYVWw.mkv,criss_cross
7,7fQiCxMwfjX0.mkv,criss_cross
8,5o2aamD_J8AM.mkv,criss_cross
9,10fQiCxMwfjX0.mkv,criss_cross


In [63]:
# df_validation.drop(index=[0,1,2], inplace=True)

In [64]:
y = pd.get_dummies(y)

In [65]:
y

,criss_cross,double_leg,rollup
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [66]:
videos_path

'./validation'

In [67]:
os.mkdir(f'{videos_path}'+'/'+"valid/")

for tag in tags:
  os.mkdir(f'{videos_path}'+'/'+"valid/"+ tag)


In [68]:
# creating two lists to store predicted and actual tags
predict = []
# predict_dict = {}
actual = []
count2 = 0
# actual = {}

for video in tqdm(df_validation.itertuples(index=False)):    
    count = 0
    videoFile = video[NAME]
    tag = video[TAG]
    cap = cv2.VideoCapture(f'{videos_path}/'+f'{tag}'+'/'+videoFile)   # capturing the video from the given path
    print(f'{videos_path}/'+f'{tag}'+'/'+videoFile)
    actual.append(tag)
    
    # actual[video[0]:video[1]]
    frameRate = cap.get(5) #frame rate
    x=1
    while cap.isOpened():
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
          # storing the frames in a new folder named train_1

          filename =f'{videos_path}'+'/valid/' +f'{tag}/'+f"video{count2}_frame%d.jpg" % count; count+=1
          cv2.imwrite(filename, frame)
    count2 += 1
    cap.release()

    prediction_images = []   
    for tag in tags:
        images = glob(f"{videos_path}/valid/{tag}/*.jpg")
      
        for i in range(len(images)):
            
            img = image.load_img(images[i], target_size=(224,224,3))
            img = image.img_to_array(img)
            # img = img/255
            prediction_images.append(img)
        


    # converting all the frames for a test video into numpy array
    # prediction_images = np.array(prediction_images)

    prediction_images = preprocess_input(np.array(prediction_images)) 
    # prediction_images = inception_v3.preprocess_input(np.array(prediction_images))   

    # extracting features using pre-trained model
    # prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    # prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
#     prediction = model.predict_classes(prediction_images)
    prediction = np.argmax(model.predict(prediction_images), axis=-1)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[stats.mode(prediction)[ONLY][ONLY]])
    print(predict, actual)

0it [00:00, ?it/s]

./validation/criss_cross/4D2Xp6LmwYPY.mkv


1it [00:02,  2.63s/it]

['criss_cross'] ['criss_cross']
./validation/criss_cross/0BSMS3ER8KqU.mkv


2it [00:03,  1.61s/it]

...

['criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'double_leg', 'double_leg', 'criss_cross', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup'] ['criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup']
./validation/rollup/0HB9ex7zlpcE.mkv


40it [01:21,  3.86s/it]

['criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'double_leg', 'double_leg', 'criss_cross', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup'] ['criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup']
./validation/rollup/7S7kw

41it [01:26,  2.10s/it]

['criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'double_leg', 'double_leg', 'criss_cross', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup'] ['criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'criss_cross', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'double_leg', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup', 'rollup']


In [69]:
tags

['criss_cross', 'double_leg', 'rollup']

In [70]:
#  ['rollup'] ['double_leg', 'rollup', 'train', 'criss_cross']

In [71]:
type(y)

pandas.core.frame.DataFrame

In [72]:
y.head()

,criss_cross,double_leg,rollup
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [73]:
# checking the accuracy of the predicted tags
accuracy_score(predict, actual)*100

36.58536585365854

In [74]:
len(predict)

41